In [7]:
# %% [markdown]
# # 3. Analítica (Capa Oro)
# Agregaciones de negocio.

# %%
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, desc, col
from delta import *

In [8]:
# Builder: puedes usar local[*] o spark://spark-master:7077 si tu cluster está ok
builder = (
    SparkSession.builder
    .appName("Lab_SECOP_Gold")
    .master("local[*]")  # <-- si quieres usar el cluster: "spark://spark-master:7077"
    .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.0.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
)

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [9]:
# %%
# Leer Plata
df_silver = spark.read.format("delta").load("data/lakehouse/silver/secop")

print("Columnas en Silver:", df_silver.columns)
print("Registros en Silver:", df_silver.count())


Columnas en Silver: ['Entidad', 'Nit_Entidad', 'Departamento', 'Ciudad', 'Estado', 'Descripcion_del_Proceso', 'Tipo_de_Contrato', 'Modalidad_de_Contratacion', 'Justificacion_Modalidad_de_Contratacion', 'Fecha_de_Firma', 'Fecha_de_Inicio_del_Contrato', 'Fecha_de_Fin_del_Contrato', 'Precio_Base', 'Valor_Total', 'Valor_Pagado']


[Stage 39:======================================>                 (34 + 4) / 50]

Registros en Silver: 1000


In [10]:
# %%
# Agregación (Shuffle)
# Usamos los nombres reales: Departamento y Precio_Base
df_gold = (
    df_silver
    .groupBy("Departamento")
    .agg(sum(col("Precio_Base").cast("double")).alias("total_contratado"))
    .orderBy(desc("total_contratado"))
    .limit(10)
)

In [11]:
# %%
# Persistir Oro
df_gold.write.format("delta").mode("overwrite").save("data/lakehouse/gold/top_deptos")

In [12]:
# %%
# Visualizar
print("Top 10 Departamentos por contratación:")
df_pandas = df_gold.toPandas()
print(df_pandas)

Top 10 Departamentos por contratación:
   Departamento  total_contratado
0         VALLE      3.941611e+11
1       BOLIVAR      3.861206e+11
2     ATLANTICO      3.845071e+11
3  CUNDINAMARCA      3.785362e+11
4     ANTIOQUIA      3.519588e+11
5     SANTANDER      3.432707e+11
6        BOGOTA      3.267886e+11
